# 0 Cargar dataframes

In [1]:
import pandas as pd

In [2]:
est_distancias = pd.read_csv('0_0_distancia_estaciones_esvelta.csv')

In [3]:
df_viajes = pd.read_csv('ecobicis-2018-03.csv')

/Users/franciscopatino/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
estaciones_lonlat = pd.read_csv('stations.csv')

# 0.1 String a List

In [5]:
# Cuando el CSV se importa, la columna con pares de retiro-arribo
# Necesita reconvertirse en una columna de listas

import ast

est_distancias['retiro_arribo'] = est_distancias.retiro_arribo.apply(lambda s: list(ast.literal_eval(s)))

# 0.2 Remover estaciones fuera de CSV longitud y latitud

In [7]:
# Para simplificar procesamiento y visualización, estaciones fuera del rango normal se remueven

df_viajes = df_viajes.drop(df_viajes[df_viajes.Ciclo_Estacion_Retiro == 1001].index)
df_viajes = df_viajes.drop(df_viajes[df_viajes.Ciclo_Estacion_Arribo == 1001].index)
df_viajes = df_viajes.drop(df_viajes[df_viajes.Ciclo_Estacion_Retiro == 3000].index)
df_viajes = df_viajes.drop(df_viajes[df_viajes.Ciclo_Estacion_Arribo == 3000].index)
df_viajes = df_viajes.drop(df_viajes[df_viajes.Ciclo_Estacion_Retiro == 1002].index)
df_viajes = df_viajes.drop(df_viajes[df_viajes.Ciclo_Estacion_Arribo == 1002].index)

df_viajes = df_viajes.reset_index(drop=True)

est_distancias = est_distancias.drop(est_distancias[est_distancias.est_retiro == 1002].index)
est_distancias = est_distancias.drop(est_distancias[est_distancias.est_arribo == 1002].index)
est_distancias = est_distancias.reset_index(drop=True)

# 1.0 Viajes ida y salida

In [8]:
# Esta función cuenta el numero de viajes de una estación hacia otra.
# Los resultados son reciprocos. Es decir, [1 a 2] es la inversa de [2 a 1] 
# para lo que son calculos de estadística de uso

from tqdm import tqdm

def fn_viajes_entrada_salida(df_viajes, df_estaciones):
    suma_salida = []
    suma_entrada = []
    for par in tqdm(df_estaciones['retiro_arribo']):
        
        suma_sal = len(df_viajes.loc[(df_viajes['Ciclo_Estacion_Retiro'] == par[0]) 
                                 & (df_viajes['Ciclo_Estacion_Arribo'] == par[1])])
    
        suma_salida.append(suma_sal)

        suma_ent = len(df_viajes.loc[(df_viajes['Ciclo_Estacion_Retiro'] == par[1]) 
                                 & (df_viajes['Ciclo_Estacion_Arribo'] == par[0])])
    
        suma_entrada.append(suma_ent)
        
    df_estaciones['trafico_entrante'] = suma_entrada
    df_estaciones['trafico_salida'] = suma_salida

In [10]:
# Prueba de función

df_rides_dict = {'Ciclo_Estacion_Retiro': [3, 4, 5, 5, 7, 8, 8],
                'Ciclo_Estacion_Arribo': [4, 3, 6, 6, 8, 7, 7]}

est_dist_dict = {'retiro_arribo': [[3, 4], [5,6], [7,8]], 
                 'distancia': [50, 70, 80], 
                 'est_retiro': [3, 5, 7], 
                 'est_arribo': [4, 6, 8]}

df_rides_dict_1_0 = pd.DataFrame(df_rides_dict)

est_dist_dict_0_1 = pd.DataFrame(est_dist_dict)

fn_viajes_entrada_salida(df_rides_dict_1_0, est_dist_dict_0_1)

est_dist_dict_0_1

100%|██████████| 3/3 [00:00<00:00, 212.29it/s]


,retiro_arribo,distancia,est_retiro,est_arribo,trafico_entrante,trafico_salida
0,"[3, 4]",50,3,4,1,1
1,"[5, 6]",70,5,6,0,2
2,"[7, 8]",80,7,8,2,1


# 1.1 Porcentaje Entradas / Salida

In [122]:
# Calculo simple del porcentaje de viajes de ida vs. regreso por pares de estaciones

# est_distancias['porcentaje'] = est_distancias['trafico_entrante'] / est_distancias['trafico_salida']

# 1.2 Coordenadas Mercator para graficar en Bokeh

In [11]:
# Sistema de coordenadas compatible con Bokeh

from ast import literal_eval
import math

def merc(Coords):
    Coordinates = literal_eval(Coords)
    lat = Coordinates[0]
    lon = Coordinates[1]
    r_major = 6378137.000
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + lat * (math.pi/180.0)/2.0)) * scale
    return (x, y)

def make_tuple_str(x, y):
    t = (x, y)
    return str(t)

In [12]:
# prueba de función 
merc(make_tuple_str(19.433571, -99.181654))

(-11040851.219314644, 2206048.982070499)

In [ ]:
######
Corre mas pruebas para encontrar método sin chaining.
######


In [13]:
# Esta función agrega coordenadas estilo mercator para visualización en Bokeh

def fn_coor_para_bokeh(dataframe):
    merc_lat = []
    merc_lon = []
   # mercator_2 = []
    for i in tqdm(range(len(dataframe))):
        mercator = merc(make_tuple_str(dataframe['lat'][i], dataframe['lon'][i]))
        merc_lat.append(mercator[0])
        merc_lon.append(mercator[1])
  #      mercator_2.append(mercator)
    
#    dict_merc = {'merc_lat': merc_lat, 'merc_lon': merc_lon}
 #   df_merc = pd.DataFrame(dict_merc)

  #  dataframe['merc_lat'] = df_merc['merc_lat']
   # dataframe['merc_lon'] = df_merc['merc_lon']
#    dataframe['mercator'] = mercator_2  
    dataframe['merc_lat'] = merc_lat
    dataframe['merc_lon'] = merc_lon

# est_mini['merc_lat'] = est_mini.apply(lambda x: merc(make_tuple_str(x['lat'], x['lon']))[0], axis = 1)
# est_mini['merc_lon'] = est_mini.apply(lambda x: merc(make_tuple_str(x['lat'], x['lon']))[1], axis = 1)
    

In [14]:
#prueba de funcion

est_mini = estaciones_lonlat.loc[estaciones_lonlat.id < 4]

fn_coor_para_bokeh(est_mini)

est_mini

100%|██████████| 3/3 [00:00<00:00, 1480.69it/s]
/Users/franciscopatino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/franciscopatino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,id,lat,lon,name,merc_lat,merc_lon
0,1,19.433571,-99.167809,1 RIO SENA-RIO BALSAS,-1.103931e+07,2.206049e+06
1,2,19.431386,-99.171695,2 RIO GUADALQUIVIR-RIO BALSAS,-1.103974e+07,2.205791e+06
2,3,19.431655,-99.158668,3 REFORMA-INSURGENTES,-1.103829e+07,2.205823e+06


In [15]:
fn_coor_para_bokeh(estaciones_lonlat)

100%|██████████| 480/480 [00:00<00:00, 12861.44it/s]


In [16]:
estaciones_lonlat.head()

,id,lat,lon,name,merc_lat,merc_lon
0,1,19.433571,-99.167809,1 RIO SENA-RIO BALSAS,-1.103931e+07,2.206049e+06
1,2,19.431386,-99.171695,2 RIO GUADALQUIVIR-RIO BALSAS,-1.103974e+07,2.205791e+06
2,3,19.431655,-99.158668,3 REFORMA-INSURGENTES,-1.103829e+07,2.205823e+06
3,4,19.428309,-99.171713,4 RIO NILO-RIO PANUCO,-1.103974e+07,2.205428e+06
4,5,19.429720,-99.169363,5 RIO PANUCO-RIO TIBER,-1.103948e+07,2.205594e+06


# 1.3 Coordenadas DF y vector

# 1.4 Vectores

In [23]:
# Función para encontrar un vector entre dos estaciones segun magnitud.
# La distancia de la flecha se aproxima al 100% entre ambos puntos según el porcentaje

import math 

def fn_vector(x0, y0, x1, y1):
    #slope 
    try:
        m = abs((y1 - y0)/(x1 - x0))
        
    except:
        m = 0
        
    dist = 1
 #   print(x0,y0,x1,y1)
    # coord Y
    if y0 > y1:
        y_3 = - m*dist*math.sqrt(1/(1+m**2))
    elif y0 < y1:
        y_3 = + m*dist*math.sqrt(1/(1+m**2))
    elif y0 == y1:
        y_3 = 0
    # coord X
    if x0 > x1:
        x_3 = - dist*math.sqrt(1/(1+m**2))
    elif x0 < x1:
        x_3 = + dist*math.sqrt(1/(1+m**2))
    elif x0 == x1:        
        x_3 = 0

        # Contingencia para coordenadas sobre la misma coordenada X
        if y0 > y1:        
            y_3 = 0 - abs((y1 - y0))
        elif y0 < y1:
            y_3 = 0 + abs((y1 - y0)) 
 
    return x_3, y_3

In [22]:
# prueba de función

fn_vector_fijo(2,2,4,4)

(0.7071067811865476, 0.7071067811865476)

In [24]:
# Esta función agrega lo que son la coordenadas de la estación salida, y coord de
# Un punto XX% entre ella y la estación destino. 
#### IMPORTANTE ####
# Como provisional, se calcula con Velocidad / Promedio velocidad total

def fn_vector_trafico(df_est_dist, df_mercator):
    est_lat_ret = []
    est_lon_ret = []
    vec_lat_arr = []
    vec_lon_arr = []
    latitude_arr_coord = []
    longitud_arr_coord = []
    porcentaje = []
    
    for i in tqdm(range(len(df_est_dist))):

        est_retiro = df_est_dist['est_retiro'][i]
        est_arribo = df_est_dist['est_arribo'][i]
        
        lat_ret = df_mercator.loc[df_mercator['id'] == est_retiro, 'merc_lat'].values[0]
        lon_ret = df_mercator.loc[df_mercator['id'] == est_retiro, 'merc_lon'].values[0]
 
        # Este try/except es para capturar a viajes donde se regresa la bicicleta a la misma
        # estación.
        
        try:
            lat_arr = df_mercator.loc[df_mercator['id'] == est_arribo, 'merc_lat'].values[0]
            lon_arr = df_mercator.loc[df_mercator['id'] == est_arribo, 'merc_lon'].values[0]
        except:
            lat_arr = lat_ret
            lon_arr = lon_ret
        est_lat_ret.append(lat_ret)
        est_lon_ret.append(lon_ret)
        latitude_arr_coord.append(lat_arr)
        longitud_arr_coord.append(lon_arr)
        
        # Este bloque es para manejar situaciones cuando una estación no registra ninguna salida.
        
        try:
            if (df_est_dist['trafico_entrante'][i] != 0):
                flujo_pct = (df_est_dist['trafico_salida'][i]) / (df_est_dist['trafico_entrante'][i] + df_est_dist['trafico_salida'][i])
            else:
                flujo_pct = 0    
        except:
            flujo_pct = 0
            
        porcentaje.append(flujo_pct)    
        lat_arr, lon_arr = fn_vector(lat_ret, lon_ret, lat_arr, lon_arr)
        vec_lat_arr.append(lat_arr)
        vec_lon_arr.append(lon_arr)
                
    df_est_dist['lat_ret'] = est_lat_ret
    df_est_dist['lon_ret'] = est_lon_ret
    df_est_dist['lat_vct'] = vec_lat_arr
    df_est_dist['lon_vct'] = vec_lon_arr
    df_est_dist['lat_arr'] = latitude_arr_coord
    df_est_dist['lon_arr'] = longitud_arr_coord
    df_est_dist['flujo_pct'] = porcentaje


In [25]:
# Prueba de función

df_rides_dict_2 = {'retiro_arribo': [[3,1], [4,2], [5,3], [6,4], [7,5], [8,6], [9,7]],
                   'est_retiro': [3, 4, 5, 6, 7, 8, 9],
                   'est_arribo': [1, 2, 3, 4, 5, 6, 7],
                   'trafico_salida': [10, 9, 8, 7, 6, 5, 4],
                   'trafico_entrante': [0, 1, 2, 3, 4, 5, 6]}

df_rides_df_1_2 = pd.DataFrame(df_rides_dict_2)

fn_vector_trafico(df_rides_df_1_2, estaciones_lonlat)

df_rides_df_1_2.head()

100%|██████████| 7/7 [00:00<00:00, 332.32it/s]


,retiro_arribo,est_retiro,est_arribo,trafico_salida,trafico_entrante,lat_ret,lon_ret,lat_vct,lon_vct,lat_arr,lon_arr,flujo_pct
0,"[3, 1]",3,1,10,0,-1.103829e+07,2.205823e+06,-0.976169,0.217013,-1.103931e+07,2.206049e+06,0.0
1,"[4, 2]",4,2,9,1,-1.103974e+07,2.205428e+06,0.005517,0.999985,-1.103974e+07,2.205791e+06,0.9
2,"[5, 3]",5,3,8,2,-1.103948e+07,2.205594e+06,0.982089,0.188416,-1.103829e+07,2.205823e+06,0.8
3,"[6, 4]",6,4,7,3,-1.103922e+07,2.205736e+06,-0.863966,-0.503549,-1.103974e+07,2.205428e+06,0.7
4,"[7, 5]",7,5,6,4,-1.104013e+07,2.205092e+06,0.789664,0.613540,-1.103948e+07,2.205594e+06,0.6


# 2.0 Ejecutando funciones

In [26]:
fn_viajes_entrada_salida(df_viajes, est_distancias)

100%|██████████| 115440/115440 [13:45<00:00, 139.87it/s]


In [28]:
df_viajes.head()

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_Estacion_Arribo,Fecha_Arribo,Hora_Arribo
0,M,49,3307,365,01/03/2018,0:00:10,169,01/03/2018,0:13:05
1,F,29,9218,137,01/03/2018,0:00:13,308,01/03/2018,0:18:34
2,F,30,3945,54,01/03/2018,0:00:17,191,01/03/2018,0:12:13
3,M,25,9971,130,01/03/2018,0:00:47,271,01/03/2018,0:12:40
4,M,20,10551,415,01/03/2018,0:00:56,433,01/03/2018,0:05:18


In [29]:
fn_vector_trafico(est_distancias, estaciones_lonlat)

  0%|          | 0/115440 [00:00<?, ?it/s]/Users/franciscopatino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
 75%|███████▍  | 86144/115440 [02:18<00:43, 666.23it/s]/Users/franciscopatino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in double_scalars
  if __name__ == '__main__':
/Users/franciscopatino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
100%|██████████| 115440/115440 [03:22<00:00, 570.51it/s]


In [31]:
fn_vector(est_distancias.loc[86129]['lat_ret'], est_distancias.loc[86129]['lon_ret'],
         est_distancias.loc[86129]['lat_arr'],est_distancias.loc[86129]['lon_arr'])
#est_distancias.loc[86129]['lat_ret']

(-0.05919822833593107, -0.9982462470562496)

In [341]:
est_distancias.loc[86129]

retiro_arribo        [239, 331]
distancia                4.6345
est_retiro                  239
est_arribo                  331
trafico_entrante              0
trafico_salida                0
porcentaje                  NaN
lat_ret            -1.10408e+07
lon_ret             2.20565e+06
lat_vct                       0
lon_vct                       0
flujo_pct                     0
lat_arr            -1.10411e+07
lon_arr             2.20073e+06
Name: 86129, dtype: object

# 3.0 Exportar

In [32]:
est_distancias.to_csv('0_1_dist_y_trafico_esvelto_4.csv', index=False)